In [1]:
import requests
import pandas as pd
import io

In [30]:
urldata2 = requests.get("https://data.cityofnewyork.us/resource/qp3b-zxtp.csv?$limit=50&$offset=150")

In [31]:
print(urldata2.status_code)

200


In [2]:
my_all_data = []

for i in range(1, 101):
    number = 0
    urldata = requests.get("https://data.cityofnewyork.us/resource/qp3b-zxtp.csv?$limit=50000&$offset=" + str(number))
    my_all_data.append(urldata)
    
    number = number + 50001

In [3]:
my_all_data_ = []

for i in my_all_data:
    thedata = i.content
    rawData_ = pd.read_csv(io.StringIO(thedata.decode('utf-8')))
    my_all_data_.append(rawData_)

In [4]:
my_all_data_[9].head()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01T00:35:40.000,2022-01-01T00:53:29.000,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01T00:33:43.000,2022-01-01T00:42:07.000,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01T00:53:21.000,2022-01-01T01:02:19.000,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01T00:25:21.000,2022-01-01T00:35:23.000,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01T00:36:48.000,2022-01-01T01:14:20.000,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [5]:
df = pd.concat(my_all_data_)

In [6]:
df.shape

(5000000, 19)

In [48]:
df.tail()

,vendorid,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,ratecodeid,store_and_fwd_flag,pulocationid,dolocationid,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
49995,2,2022-01-01T19:14:47.000,2022-01-01T19:49:51.000,1.0,17.48,2.0,N,79,132,1,52.0,0.0,0.5,1.00,6.55,0.3,64.10,2.5,1.25
49996,2,2022-01-01T19:09:24.000,2022-01-01T19:13:37.000,1.0,0.77,1.0,N,161,164,1,4.5,0.0,0.5,1.95,0.00,0.3,9.75,2.5,0.00
49997,2,2022-01-01T19:57:43.000,2022-01-01T20:08:56.000,1.0,3.07,1.0,N,79,256,1,11.5,0.0,0.5,2.96,0.00,0.3,17.76,2.5,0.00
49998,2,2022-01-01T19:02:24.000,2022-01-01T19:05:39.000,3.0,0.86,1.0,N,170,164,2,4.5,0.0,0.5,0.00,0.00,0.3,7.80,2.5,0.00
49999,2,2022-01-01T19:22:29.000,2022-01-01T19:34:10.000,2.0,2.22,1.0,N,170,236,1,10.0,0.0,0.5,3.32,0.00,0.3,16.62,2.5,0.00


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.appName("nyc_yellow_taxi_trip").getOrCreate()

24/03/14 10:21:40 WARN Utils: Your hostname, MKM.local resolves to a loopback address: 127.0.0.1; using 192.168.88.77 instead (on interface en1)
24/03/14 10:21:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 10:21:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sparkContext

<SparkContext master=local[*] appName=nyc_yellow_taxi_trip>

In [50]:
df_spark = spark.read.csv("/Users/mikemusas/Downloads/2022_Yellow_Taxi_Trip_Data.csv", header=True, inferSchema=True)

In [51]:
df_spark_n = (df_spark
 .withColumn("pickup_datetime", to_timestamp(col("tpep_pickup_datetime"), "MM/dd/yyyy hh:mm:ss a"))
 .drop("tpep_pickup_datetime")
 .withColumn("dropoff_datetime", to_timestamp(col("tpep_dropoff_datetime"), "MM/dd/yyyy hh:mm:ss a"))
 .drop("tpep_dropoff_datetime")
 )

In [6]:
df_spark_n.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)


In [7]:
df_spark_n.show(n=3, truncate=False)

+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------------+-------------------+
|VendorID|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|pickup_datetime    |dropoff_datetime   |
+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------------+-------------------+
|1       |2              |3.8          |1         |N                 |142         |236         |1           |14.5       |3.0  |0.5    |3.65      |0.0         |0.3                  |21.95       |2.

In [52]:
df_spark_22 = (df_spark_n
               .select("*")
               .where((year(col("pickup_datetime")) == 2022) & (month(col("pickup_datetime")) == 1))
               )

In [53]:
df_spark_22.show(n=3, truncate=False)

+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------------+-------------------+
|VendorID|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|pickup_datetime    |dropoff_datetime   |
+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------------+-------------------+
|1       |2              |3.8          |1         |N                 |142         |236         |1           |14.5       |3.0  |0.5    |3.65      |0.0         |0.3                  |21.95       |2.

In [23]:
(df_spark_22
 .select(month(col("pickup_datetime")))
 .distinct()
 .show(n=12, truncate=False)
 )

+----------------------+
|month(pickup_datetime)|
+----------------------+
|1                     |
|3                     |
|2                     |
+----------------------+


In [29]:
df_spark_22.count()

9071207

In [35]:
(df_spark_22
 .select(month("pickup_datetime"))
 .groupBy(month(col("pickup_datetime")))
 .count()
 .show(truncate=False)
 )

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `pickup_datetime` cannot be resolved. Did you mean one of the following? [`month(pickup_datetime)`].;
'Aggregate [month('pickup_datetime)], [month('pickup_datetime) AS month(pickup_datetime)#1424, count(1) AS count#1423L]
+- Project [month(cast(pickup_datetime#892 as date)) AS month(pickup_datetime)#1419]
   +- Filter ((year(cast(pickup_datetime#892 as date)) = 2022) AND (month(cast(pickup_datetime#892 as date)) <= 3))
      +- Project [VendorID#854, passenger_count#857, trip_distance#858, RatecodeID#859, store_and_fwd_flag#860, PULocationID#861, DOLocationID#862, payment_type#863, fare_amount#864, extra#865, mta_tax#866, tip_amount#867, tolls_amount#868, improvement_surcharge#869, total_amount#870, congestion_surcharge#871, airport_fee#872, pickup_datetime#892, dropoff_datetime#933]
         +- Project [VendorID#854, passenger_count#857, trip_distance#858, RatecodeID#859, store_and_fwd_flag

In [ ]:
(df_spark
 .select("Color", "Count")
 .groupBy("Color")
 .avg("Count")
 .orderBy("avg(Count)", ascending=False)
 .show(truncate=False)
 )

In [54]:
# Save a Parquet file
parquet_path = "data_january/"
df_spark_22.write.format("parquet").save(parquet_path)

In [48]:
# Reading the Parquet file
parquet_df = spark.read.parquet(parquet_path)

In [49]:
parquet_df.show(n=10, truncate=False)

+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------------+-------------------+
|VendorID|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|pickup_datetime    |dropoff_datetime   |
+--------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------------+-------------------+
|2       |2              |2.5          |1         |N                 |140         |162         |1           |19.0       |0.0  |0.5    |4.46      |0.0         |0.3                  |26.76       |2.